In [1]:
### This file does a RZ gate (1 parameter, 1 qubit) with X,Y control
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

In [2]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units
;

In [3]:
# Operators
const Paulis = Dict(
    "I" => Matrix{ComplexF64}([1 0; 0 1]),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 im; -im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
const a = [0 1; 0 0]
const ad = transpose(a)
excitation(theta,phi) = exp(-im/2 * theta * Paulis["Z"]) * exp(-im/2 * theta * Paulis["Y"]);  


H_drives = [
     Paulis["X"],
     Paulis["Y"]
]
system = QuantumSystem(H_drives);
t_f = 10 * ns
n_steps = 51
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
Δt = times[2] - times[1]
n_controls=2
n_qubits=1;

In [4]:
### Generate Initial Trajectories 
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 1000.
R =1e-2
R_b=200.0
# Add control bounds
a_bound = 1.0
dda_bound = 1.0

ops = Options()
ops.print_info_string = "yes"
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1.0 ##down
ops.print_level = 0
p=UnitarySmoothPulseProblem(
                system,
                excitation(pi,pi),
                n_steps,
                Δt;
                a_bound=a_bound,
                dda_bound=dda_bound,
                Q=Q,
                R=R,
                verbose=true,
                hessian_approximation=true,
                pade_order=10,
                free_time=true,
                timesteps_all_equal=true,
                max_iter=PICO_max_iter,
                ipopt_options=ops,
                )
min_time_problem  = UnitaryMinimumTimeProblem(p; final_fidelity=1-1e-5,max_iter=500);
solve!(min_time_problem)

    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
applying constraint: time step all equal constraint
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
applying constraint: bounds on Δt
applying constraint: time step all equal constraint

******************************************************************************
This pro

In [5]:
Δt = ceil(1.2 * min_time_problem.trajectory[:Δt][1]*100)/100

0.13

In [ ]:
N = 11
trajectory_list,glued_trajectory_list = Interpolator2D(
    excitation,
    system,
    n_steps,
    N,
    Δt;
    Q=Q,
    R=R,
    R_b=R_b,
    a_bound=a_bound,
    dda_bound=dda_bound,
    ipopt_options=ops,
    max_iter=PICO_max_iter,
    fidelity_constraint = 1e-5
);

    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a
applying constraint: final value of a
applying constraint: bounds on a
applying constraint: bounds on dda
    building dynamics from integrators...
        constructing knot point dynamics functions...
        determining dynamics derivative structure...
            computing jacobian sparsity...
            creating full trajectory jacobian structure...
        constructing full dynamics derivative functions...
    building evaluator...
    initializing optimizer...
applying constraint: initial value of Ũ⃗
applying constraint: initial value of a


In [ ]:
DATA = sample2D(trajectory_list,Δt,n_qubits,system,50,N,excitation)
findmax(DATA)

In [ ]:
using Plots
gr()
Plots.heatmap(range(0,2*pi,50),
    range(0,2*pi,50), log10.(DATA),
    xlabel="θ values", ylabel="ϕ values",
    title="Log Infidelity")

In [ ]:
DATA = simple_sample2D(glued_trajectory_list,Δt,n_qubits,system,50,N,excitation)
findmax(DATA)

In [ ]:
gr()
Plots.heatmap(range(0,2*pi,50),
    range(0,2*pi,50), log10.(DATA),
    xlabel="θ values", ylabel="ϕ values",
    title="Log Infidelity")

In [ ]:
data,d_data,dd_data = parse2D(trajectory_list,2,N);

In [ ]:
verify2D(dd_data,n_qubits,system,Δt,n_steps,N,excitation)

In [ ]:
verify2D([[glued_trajectory_list[i][j][:dda] for j in 1:N] for i in 1:N],n_qubits,system,Δt,n_steps,N,excitation)

In [ ]:
export_csv2D([[glued_trajectory_list[i][j][:dda] for j in 1:N] for i in 1:N],n_controls,N,"XYControlA.ipynb_accels.csv")